<a href="https://colab.research.google.com/github/iued-uni-heidelberg/DAAD-Training-2021/blob/main/Terminologieextraktion5KeynessScoresV01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The analysis of keyness, which will be used in term extraction

---
We will also prepare the gold standard evaluation set for the project


In [ ]:
# Stage 0: Preparing Gold standard: Reading / extracting information from gold standard: creating a list of annotated terms
!wget https://heibox.uni-heidelberg.de/f/ae1110c4f9ad42b9a3d5/?dl=1
!mv index.html?dl=1 BGH1_s00Astghik.txt
!wget https://heibox.uni-heidelberg.de/f/0c787f26123f49178639/?dl=1
!mv index.html?dl=1 BGH1_s00Hayk.txt

In [2]:
!cat BGH1_s00Astghik.txt BGH1_s00Hayk.txt >BGH1_s00GoldStandard.txt

In [3]:
FInput = open('BGH1_s00GoldStandard.txt', 'r')
FOutput = open('BGH1_s00GoldS_Terms.txt', 'w')
# for statistical purposes - separately single and multiword terms
FOutputDict1w = open('BGH1_s00GoldS_D1w.txt', 'w') # 1-word terms
FOutputDict2w = open('BGH1_s00GoldS_D2w.txt', 'w') # 2-word terminological expressions
FOutputDictMWE = open('BGH1_s00GoldS_DMWE.txt', 'w') # more than 2 words

In [4]:
def printDictionary(DictionaryFrq, FOut, Rev = True):
    for Word, Frq in sorted( DictionaryFrq.items() , key=lambda x: x[1], reverse=Rev):
        FOut.write(Word + '\t' + str(Frq) + '\n')
    return

In [5]:
import re, os, sys
LGSTerms = [] # gold standard terms
DGS1w = {} # dictionary of single words
DGS2w = {} # dictionary of single words
DGSMWE = {} # dictionary of mwes
IGS1w = 0 # number of annotated tokens of single words
IGS2w = 0
IGSMWE = 0 # number of annotated tokens of multiwords
for SLine in FInput:
    LAnnotatedTermsInLine = re.findall('<<([^><]+)>>', SLine)
    LGSTerms.extend(LAnnotatedTermsInLine)

for GSTerm in LGSTerms:
    GSTerm = GSTerm.strip()
    GSTerm = re.sub(' +', ' ', GSTerm)
    LGSTErms = re.split(' ', GSTerm)
    if len(LGSTErms) > 2:
        IGSMWE += 1
        try: DGSMWE[GSTerm] += 1
        except: DGSMWE[GSTerm] = 1
    if len(LGSTErms) == 2:
        IGS2w += 1
        try: DGS2w[GSTerm] +=1
        except: DGS2w[GSTerm] = 1
    else:
        IGS1w += 1
        try: DGS1w[GSTerm] +=1
        except: DGS1w[GSTerm] = 1

    FOutput.write(GSTerm + '\n')

FOutputDictMWE.write('# Number of tokens: ' + str(IGSMWE) + '\n')
FOutputDict2w.write('# Number of tokens: ' + str(IGS2w) + '\n')
FOutputDict1w.write('# Number of tokens: ' + str(IGS1w) + '\n')

printDictionary(DGSMWE, FOutputDictMWE)
printDictionary(DGS2w, FOutputDict2w)
printDictionary(DGS1w, FOutputDict1w)

FOutputDictMWE.flush()
FOutputDictMWE.close()
FOutputDict2w.flush()
FOutputDict2w.close()
FOutputDict1w.flush()
FOutputDict1w.close()

FOutput.flush()
FOutput.close()

FInput.close()

## Testing for keyness -- the gold standard

Here we cross-check the gold standard in terms of the keywords, which have been identified in comparison with Wikipedia and Europarl-7 corpora


In [6]:
!wget https://heibox.uni-heidelberg.de/f/aa4560e627bd4b1d8055/?dl=1
!mv index.html?dl=1 TK_KW_Verif_V02.csv

--2021-10-26 19:29:03--  https://heibox.uni-heidelberg.de/f/aa4560e627bd4b1d8055/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/ef8547fb-ce0e-49e8-a2af-e8860fcb64ac/TK_KW_Verif_V02.csv [following]
--2021-10-26 19:29:04--  https://heibox.uni-heidelberg.de/seafhttp/files/ef8547fb-ce0e-49e8-a2af-e8860fcb64ac/TK_KW_Verif_V02.csv
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 38812 (38K) [application/octet-stream]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>]  37.90K   139KB/s    in 0.3s    

2021-10-26 19:29:05 (139 KB/s) - ‘index.html?dl=1’ saved [38812/38812]



In [7]:
FInputKW = open('TK_KW_Verif_V02.csv', 'r')
FOutputGSKWS1w = open('BGH1_s01GoldSKW_D1w.txt', 'w')
FOutputGSKWS2w = open('BGH1_s01GoldSKW_D2w.txt', 'w')
FOutputGSKWSMWE = open('BGH1_s01GoldSKW_MWE.txt', 'w')

In [8]:
DScoresKW = {} # keywords - scores
DScoresNK = {} # non-keywords
DStatusKW = {} # status: key/non-key-word
for Line in FInputKW:
    LFieldsKW = re.split('\t', Line)
    SWord = LFieldsKW[1]
    AKScore = float(LFieldsKW[2])
    AKStatus = float(LFieldsKW[3])
    DStatusKW[SWord] = AKStatus
    if AKStatus > 0:
        DScoresKW[SWord] = AKScore
    else:
        DScoresNK[SWord] = AKScore

In [9]:
# first we compile match patterns for all keywords and store them in a dictionary
DScoresREKW = {} # dictionary of RE
for kw, val in DScoresKW.items():
    RPatternKW = re.compile('^' + kw + '$', re.IGNORECASE)
    DScoresREKW[RPatternKW] = val # we convert all keywords to RE search patterns, with the ignore case flag



In [ ]:
matchedMWEs = 0
for STermGS, frq in DGSMWE.items(): # freq dictionary of gold standard terms annotated in the document
    LAGoldSKeywordValues = []
    LTermsGS = re.split(' ', STermGS)
    m = 0
    for STermWordGS in LTermsGS:
        for REkw, val in DScoresKW.items():
            if re.match(REkw, STermWordGS):
                m+=1
                LAGoldSKeywordValues.append(val) # values of keyness
                print(STermGS + ' ~ ' + STermWordGS + ' ' + str(val))
    if m>0: matchedMWEs += 1
    # print(m)
    # print(matchedMWEs)

print(matchedMWEs)

A quicker algorithm to do the same check: we convert to all possible string variants (upper, lower, sentence case and just look up the dictinary)

In [12]:
# alternative: we create a dictionary with lower, upper and sentence case variants
DScoresREKWquick = {} # dictionary of RE
for kw, val in DScoresKW.items():
    SUpperC = kw.upper()
    SLowerC = kw.lower()
    SSentenceC = kw.capitalize()
    # RPatternKW = re.compile('^' + kw + '$', re.IGNORECASE)
    DScoresREKWquick[SUpperC] = val # 
    DScoresREKWquick[SLowerC] = val # 
    DScoresREKWquick[SSentenceC] = val # 


In [ ]:
# 2nd variant: with quick dictionary
matchedMWEs = 0
for STermGS, frq in DGSMWE.items(): # freq dictionary of gold standard terms annotated in the document
    LAGoldSKeywordValues = []
    LTermsGS = re.split(' ', STermGS)
    m = 0
    for STermWordGS in LTermsGS:
        if STermWordGS in DScoresREKWquick:
            m+=1
            val = DScoresREKWquick[STermWordGS]
            LAGoldSKeywordValues.append(val)
            print(STermGS + ' ~ ' + STermWordGS + ' ' + str(val))

    if m>0: matchedMWEs += 1
    # print(m)
    # print(matchedMWEs)

print(matchedMWEs)